In [1]:
import sys
sys.path.append('../')
from ftx_utilities import *
%matplotlib inline

## benchmark

In [2]:
benchmark=pd.read_pickle('DONOTDELETE_cost_blind/ladder.pickle')

In [25]:
concentration_limit=9
slippage_override=2-4
benchmark['totalPnL']=(benchmark['RealizedCarry']+benchmark['transactionCost'])
benchmark['trackingError']=(benchmark['RealizedCarry']-benchmark['ExpectedCarry'])
perfs=benchmark.groupby(by=['name'])['absWeight'].describe().sort_values(by='mean',key=abs,ascending=False).head(7)/1e5
mains=benchmark[benchmark['name'].isin(list(perfs.index)+['total','USD'])]
perfs

,count,mean,std,min,25%,50%,75%,max
name,,,,,,,,
USD,0.00011,1.682775,8.519542e-01,4.232281e-01,1.111650,1.731998,2.251437,2.878533
total,0.00011,1.000000,2.108773e-16,1.000000e+00,1.000000,1.000000,1.000000,1.000000
MKR-PERP,0.00011,0.370907,1.788205e-01,5.865345e-09,0.381720,0.449318,0.477998,0.500000
ALPHA-PERP,0.00011,0.367229,1.411879e-01,1.512381e-01,0.232582,0.429517,0.490605,0.500000
BNT-PERP,0.00011,0.288583,1.466774e-01,5.831747e-02,0.202752,0.291506,0.388814,0.496664
SUSHI-PERP,0.00011,0.274534,1.453680e-01,2.129406e-17,0.208121,0.291568,0.393998,0.418496
GRT-PERP,0.00011,0.271286,1.921675e-01,0.000000e+00,0.099581,0.321535,0.406498,0.500000


In [29]:
txcost=mains.groupby(by=['name'])['transactionCost'].describe()
txcost

,count,mean,std,min,25%,50%,75%,max
name,,,,,,,,
ALPHA-PERP,11.0,-187.188984,212.096823,-671.814277,-241.081879,-107.314807,-42.215114,-0.000031
BNT-PERP,11.0,-394.800543,367.342309,-1158.522568,-548.041017,-239.342335,-110.090095,-25.314492
GRT-PERP,11.0,-538.025012,618.512966,-1828.924292,-540.761511,-301.567255,-172.613509,-2.942960
MKR-PERP,11.0,-330.025461,540.517316,-1832.343687,-227.651788,-118.389359,-87.641923,-0.985336
SUSHI-PERP,11.0,-190.281141,163.154508,-588.415713,-259.507708,-155.236725,-83.778937,-7.127771
USD,11.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
total,11.0,-5638.810344,3140.985766,-12511.108113,-7282.672637,-5607.904975,-3604.568249,-983.734656


In [30]:
absWeight=mains.groupby(by=['name'])['optimalWeight'].describe()
absWeight

,count,mean,std,min,25%,50%,75%,max
name,,,,,,,,
ALPHA-PERP,11.0,36722.900896,1.411879e+04,1.512381e+04,23258.157335,42951.746791,49060.472112,50000.000000
BNT-PERP,11.0,-28858.324059,1.466774e+04,-4.966637e+04,-38881.411436,-29150.602188,-20275.215341,-5831.747409
GRT-PERP,11.0,-3992.690062,3.407789e+04,-4.999996e+04,-35554.603540,0.000000,22661.159789,42343.845850
MKR-PERP,11.0,12276.595373,4.083222e+04,-4.629030e+04,-22295.777815,35725.966274,47120.526787,49999.999991
SUSHI-PERP,11.0,27453.392363,1.453680e+04,-2.129406e-12,20812.103010,29156.844622,39399.840548,41849.585268
USD,11.0,-49514.443446,1.889721e+05,-2.878533e+05,-209863.712251,-42322.806407,111164.968238,226519.318273
total,11.0,100000.000000,2.108773e-11,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000


In [43]:
mains.pivot(index='time',columns='name',values='RealizedCarry').iplot()

In [32]:
mains.loc[mains['name']=='total',['trackingError','totalPnL','time','transactionCost','excessIM','excessMM']].set_index('time').iplot()

In [ ]:
a=compile_runs()

## todo
# - tx cost = 0 for benchmark, 5bps
# - realized vs expected fit -> finer hold/sig
# - raise the bar for volume, blowup
# - munraj

## Comparative run statistics

In [ ]:
c=0.5
all=1*a.xs([c,'RealizedCarry'],
       level=['concentration_limit','field'])\
    +a.xs([c,'transactionCost'],
       level=['concentration_limit','field'])
all['total'].groupby(by=['holding_period','signal_horizon']).describe()*100

## benchmark: transaction costs block rebalance ?

In [ ]:
benchmark=a.xs([c,timedelta(hours=1),timedelta(hours=1)],
       level=['concentration_limit','holding_period','signal_horizon']).reorder_levels(['field','time'])

## Zoom on a run

In [ ]:
run=a.xs([c,timedelta(days=2),timedelta(days=7)],
       level=['concentration_limit','holding_period','signal_horizon']).reorder_levels(['field','time'])

In [ ]:
relevant=(run.T['absWeight'].T).max().sort_values(ascending=False)
relevant=relevant[relevant>0.02]
run=run[relevant.index]

In [ ]:
fig=px.line(run.loc['optimalWeight'])
fig.show()

In [ ]:
fig=px.line(run.loc['RealizedCarry'])
fig.show()

In [ ]:
fig=px.line(run.loc['ExpectedCarry'])
fig.show()